In [5]:
import json
import sys
from utils import load_config, fetch_api_data

In [6]:
#Load config file
config = load_config('config.json')

#Get API key
weather_api_key = config.get('National Centers for Environmental Information', {}).get('key')
if not weather_api_key:
    print("Weather API key not found in the configuration file.")
    sys.exit()

# Set the parameters for the type of place
dataset = "global-summary-of-the-year"
dataTypes = ["DP01","DP05","DP10","DSND","DSNW","DT00","DT32","DX32","DX70","DX90","SNOW","PRCP"]
stations = "ASN00084027"
startDate = "1960-01-01"
endDate = "1960-12-31"
units = "standard"
includeAttributes = "true"
format = "json"


# set up a parameters dictionary
params = {
    "dataset":dataset,
    "dataTypes":dataTypes,
    "stations":stations,
    "startDate":startDate,
    "endDate":endDate,
    "units":units,
    "includeAttributes":includeAttributes,
    "format":format
}

# Set base URL
base_url = f"https://www.ncei.noaa.gov/access/services/data/v1"

# run a request using our params dictionary
data = fetch_api_data(base_url, params) # type: ignore
print(json.dumps(data, indent=2))

[
  {
    "DATE": "1960",
    "STATION": "ASN00084027",
    "PRCP_ATTRIBUTES": " ,a",
    "PRCP": "45.19"
  }
]
